In [1]:
import os
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
#import pickle, datetime
import math

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

import  PIL.Image

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
root_path = '/Users/jdu12/Desktop/research/'
model_dir = root_path + 'saved_models/AlexNet/'
train_dir = root_path + 'train/'
val_dir = root_path + 'val/'

## Data Loading

In [3]:
N_CATEGORY = 36
BATCH_SIZE = 128


train_datagen = ImageDataGenerator(horizontal_flip=False)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=BATCH_SIZE,
    class_mode='categorical')


val_datagen = ImageDataGenerator(horizontal_flip=False)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(227, 227),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

Found 4229 images belonging to 36 classes.
Found 846 images belonging to 36 classes.


## AlexNet Architecture 

### Model Parameters 

In [4]:
DROPOUT = 0.5
model_input = Input(shape = (227, 227, 3))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)
#model.summary()

In [5]:
WEIGHT_DECAY = 0.0005

loss_metric = "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=WEIGHT_DECAY, amsgrad=False))

routine_dir = model_dir + "routine-{epoch:02d}-{val_acc:.2f}.hdf5"
save_models = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
best_dir = model_dir + "best-{epoch:02d}-{val_acc:.2f}.hdf5"
save_best = ModelCheckpoint(best_dir, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
res = model.fit_generator(
    train_generator, steps_per_epoch = math.ceil(4229/BATCH_SIZE),
    epochs=190, verbose=2, callbacks=[save_models, save_best], 
    validation_data=val_generator, validation_steps = math.ceil(846/BATCH_SIZE)
)

Epoch 1/190
 - 12s - loss: 3.2609 - acc: 0.1274 - val_loss: 3.3525 - val_acc: 0.0709
Epoch 2/190
 - 8s - loss: 1.4270 - acc: 0.4652 - val_loss: 4.8317 - val_acc: 0.0378
Epoch 3/190
 - 9s - loss: 0.9107 - acc: 0.6470 - val_loss: 2.4089 - val_acc: 0.3132
Epoch 4/190
 - 9s - loss: 0.7889 - acc: 0.7300 - val_loss: 1.7905 - val_acc: 0.3156
Epoch 5/190
 - 9s - loss: 0.6026 - acc: 0.7912 - val_loss: 4.4019 - val_acc: 0.1064
Epoch 6/190
 - 9s - loss: 0.3999 - acc: 0.8641 - val_loss: 0.5278 - val_acc: 0.8132
Epoch 7/190
 - 9s - loss: 0.3133 - acc: 0.8954 - val_loss: 0.3186 - val_acc: 0.9019
Epoch 8/190
 - 9s - loss: 0.3905 - acc: 0.8710 - val_loss: 0.8140 - val_acc: 0.7092
Epoch 9/190
 - 9s - loss: 0.3446 - acc: 0.8876 - val_loss: 1.1799 - val_acc: 0.6040
Epoch 10/190
 - 9s - loss: 0.2599 - acc: 0.9248 - val_loss: 0.7012 - val_acc: 0.7624
Epoch 11/190
 - 9s - loss: 0.1856 - acc: 0.9510 - val_loss: 0.2734 - val_acc: 0.9267
Epoch 12/190
 - 9s - loss: 0.1976 - acc: 0.9430 - val_loss: 2.1142 - val_

Epoch 96/190
 - 8s - loss: 0.0022 - acc: 0.9993 - val_loss: 0.3922 - val_acc: 0.9326
Epoch 97/190
 - 7s - loss: 0.0037 - acc: 0.9986 - val_loss: 0.2855 - val_acc: 0.9515
Epoch 98/190
 - 7s - loss: 0.0018 - acc: 0.9995 - val_loss: 0.2882 - val_acc: 0.9634
Epoch 99/190
 - 7s - loss: 0.0055 - acc: 0.9977 - val_loss: 0.2938 - val_acc: 0.9574
Epoch 100/190
 - 8s - loss: 0.0062 - acc: 0.9979 - val_loss: 0.6109 - val_acc: 0.9078
Epoch 101/190
 - 7s - loss: 0.0061 - acc: 0.9982 - val_loss: 0.3658 - val_acc: 0.9444
Epoch 102/190
 - 7s - loss: 0.0117 - acc: 0.9920 - val_loss: 0.6925 - val_acc: 0.8688
Epoch 103/190
 - 8s - loss: 0.2742 - acc: 0.9420 - val_loss: 4.0088 - val_acc: 0.2482
Epoch 104/190
 - 8s - loss: 0.0463 - acc: 0.9839 - val_loss: 3.1113 - val_acc: 0.3806
Epoch 105/190
 - 7s - loss: 0.0248 - acc: 0.9922 - val_loss: 0.7230 - val_acc: 0.8014
Epoch 106/190
 - 8s - loss: 0.0147 - acc: 0.9954 - val_loss: 0.4424 - val_acc: 0.8901
Epoch 107/190
 - 8s - loss: 0.0187 - acc: 0.9947 - val_los

In [ ]:
'''
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))
'''